## 組合せ最適化問題
量子コンピュータで最適化問題を解くには、イジングモデルといわれる物理モデルを利用する。

## QUBO定式化
QUBOは問題の答えが小さいほうが正解になるように設定された式です。式の形は、

$$
QUBO = -\sum_i h_i q_i -\sum_{i,j}J_{ij}q_iq_j
$$

となっている。iとjは点を表し、hはバイアス（局所磁場）、Jは相互作用と呼ばれます。この式ではqは量子ビットを表し0か1を取ります（イジングの場合は+1か-1）。
私たちはhとJを問題として設定し、qの値を求めます。

## なぜ最小化できるのか？
QUBO式は、変数$q$が変化することで最小化されます。  
$$
QUBO=Q(q_1,q_2,...,q_n)
$$
ここで, $J_{i,j}$と$h_i$は解きたい問題により決まります。

変数$q_i$を時間の関数として、$q_i(t)$の更新式は、  
$$q_i(t+1)=q_i(t)- \sum_{j}J_{ij}q_j(t)$$
のように, $q_i$を更新していくと、QUBO関数は最小化することが証明されている。  

$$
\frac{dQ(q_i)}{dt} \leq 0
$$

$\sum_{j}J_{ij}q_j$の部分は、隣接する変数の状態に$J_{i,j}$の重みを付けて総和を取る

## 問題設定の仕方
問題の設定の仕方は、グラフ問題というものに問題を落とすことで計算できますが、いくつか問題を解くことでコツをつかめます。

主に問題のコスト関数は二種類の式を考える必要があります。

１．小さくしたいコスト関数  
２．満たすべき条件（制約条件）

この二つを別々に設計し、つなげることで実装できます。片方しかない式もあります。

## 使うツール

networkx（ネットワークグラフを書く）  
matplotlib（各種のグラフを書く）  
numpy（数値ライブラリ）  

In [1]:
!pip install --quiet networkx matplotlib

## ジョブシーケンス問題
実行時間が知られているジョブがN個あります.  
ノード数がM個のコンピュータに行わせるときに, 最大実行時間のノードの実行時間を最小化する問題.  
<img src="./img/job2.png" width=300>

上のジョブをコンピューターに分けて計算するときに、各コンピューターの実行時間が同じになるようにジョブを配分する。
<img src="./img/job.png" width=800>

## QUBO式
<img src="./img/job3.png" width=800>

バイナリ変数は$\alpha$番目のノードで$i$番目のジョブを実行するときに$1$, そうでない場合は$0$をとる.  
$$
\Large
x_{\alpha,i}
$$

$i$番目のジョブの実行時間を$L_i$とする. $(0 \leqq i \leqq N)$

コスト関数は, 実行時間が最大のノードの実行時間を最小化したい. 
$$
\large
\sum_{i} L_i x_{\alpha,i}
$$
工夫として, 最大実行時間となるノードを1番目に固定して定式化する. 
1番目のノードの実行時間を定式化すると以下になり, これを最小化する. 
$$
\large
\sum_{i} L_i x_{1,i}
$$


ノード1が最大実行時間のノードである条件は, ノード1とノード$\alpha$の実行時間との差が, $>0$. 
$$
\large
\sum_{i} L_i(x_{1,i} - x_{\alpha,i}) > 0
$$


不等式をQUBO式で扱うために補助変数を用意する.  
ノード$\alpha$とノード1の実行時間との差が,$n$のときに$y_{n,\alpha}=1$とする. 
<img src="./img/job6.png" width=800>
$$
\large
y_{n,\alpha}
$$

ノード$\alpha$とノード1の実行時間との差が,$n$である条件式
$$
\large \sum_n^M n y_{n,\alpha} = \sum_{i} L_i x_{1,i} - L_i x_{\alpha,i}
$$
定数$M$はノード1との実行時間の差の最大値で, 任意で決めるハイパーパラメータ

コスト関数
$$
\large H_A = \sum_{i} L_i x_{1,i}
$$
制約式  

ジョブ$i$は, ただ1つのノードのみで実行される条件式
$$
\large H_B = \sum_{i} (1 - \sum_{\alpha} x_{\alpha,i})^2 
$$

ノード1とノード$\alpha$との実行時間の差の関係式
$$
\large H_C = \sum_{\alpha=2} (\sum_n^M n y_{n,\alpha}- \sum_{i} L_i x_{1,i} - L_i x_{\alpha,i})^2+ \sum_n^M ( 1 - y_{n,\alpha})^2 
$$

QUBO式
$$
\large H=H_A+H_B+H_C  
$$

$$
\large
=\sum_{i} L_i x_{1,i} + \sum_{i} 1 - \sum_{\alpha} x_{\alpha,i})^2 +\sum_{\alpha=2} (\sum_n^M n y_{n,\alpha}- \sum_{i} (L_i x_{1,i} - L_i x_{\alpha,i})^2+ \sum_n^M ( 1 - y_{n,\alpha})^2 
$$

### LOGエンコーディング

$y_{n,\alpha}$は$\alpha$に対して, 1つだけ1でその他は0になるので,変数の無駄が多くなります. 
ノード1との実行時間の差を2進数でエンコードすると$N$を表現する2進数は$log_2(M-1)+1$個の変数で実現できます. 
このエンコード方法をLOGエンコードと呼びます. 
$y_n$がただ1つ1になる最小の方法はone-hotエンコーディングと呼びます. 

<img src="./img/job5.png" width=500>

LOGエンコーディングの変数を$z_{n,\alpha}$として, ノード1との実行時間との差を2進数で表す。

$$
\large z_{n,\alpha}
$$

LOGエンコーディングを用いる場合は、次の制約を満たすように$z_{\alpha,i}$を追加する。
$$
\large M > \sum_{i} L_i x_{1,i} - L_i x_{\alpha,i}
$$

不等式制約を等式にするために$z_{\alpha,i}$を追加
$$
\large M -\sum_{i} L_i x_{1,i} - L_i x_{\alpha,i} -z_{\alpha,i}
$$

$$
\large QUBO=\sum_{i} L_i x_{1,i} + \sum_{i} (1 - \sum_{\alpha} x_{\alpha,i})^2 +\sum_{\alpha=2} (M- \sum_{i} L_i x_{1,i} - L_i x_{\alpha,i}-\sum_{n=0}^{log_2(M-1)} 2^n z_{n,\alpha})^2 
$$

## 問題設定
$10$個のジョブがある. 
各ジョブの実行時間は, 

|ジョブ |  1   |  2   |  3   |  4   |  5   |  6   |  7   |  8   |  9   | 10   |
| ----  | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| 時間  |  1   |  2   |  3   |  4   |  5   |  6   |  7   |  8   |  9   | 10   |

上記のジョブを3つのノードに分けて実行するときに, 最大実行時間のノードを最小化する. 

| ノード|  1   |  2   |  3   |  4   | 合計 |
| ----  | ---- | ---- | ---- | ---- | ---- |
|ノード1|  4   |  7   |  8   |  0   |  19  |
|ノード2|  1   |  2   |  5   |  10  |  18  |
|ノード3|  3   |  6   |  9   |  0   |  18  |

In [2]:

import numpy as np

#今回は4量子ビットのみ
J=10 #ジョブの数
N=3  #ノードの数
M=3  #最大実行時間の差

#q = Array.create('q', shape=(N,J), vartype='BINARY')
z = LogEncInteger("z", (0, M-1))

#ジョブの実行時間
L=np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])


#式A
HA=0
for i in range(J):
   HA += L[i]*q[0,i]

#式B
HB=0
for i in range(J):
    tmp=0
    for a in range(N):
        tmp += q[a,i]
    HB += (1-tmp)**2

#式C
HC=0
for a in range(1,N):
    tmp=0
    for i in range(J):
        tmp += L[i]*(q[0,i]-q[a,i])
    HC += (M-tmp-z)**2

#式をつなげる
H = HA + 5*HB + 2*HC

The code failed because of a fatal error:
	Error sending http request and maximum retry encountered..

Some things to try:
a) Make sure Spark has enough available resources for Jupyter to create a Spark context.
b) Contact your Jupyter administrator to make sure the Spark magics library is configured correctly.
c) Restart the kernel.


ノード0 実行ジョブ3,6,7     時間=16  
ノード1 実行ジョブ0,1,4,9   時間=15  
ノード2 実行ジョブ2,5,8     時間=15  

## 4-2. ナップザック問題
価値が分かっている$N$個の荷物があります. この荷物をナップザックの最大容量$W$を超えない範囲で入れるときに, 価値の合計を最大化する組み合わせを探す問題です. 

| 荷物 |  1   |  2   |  3   |  4   |  5   |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 重さ |  4   |  7   |  1   |  3   |  5   |
| 価値 |  10  |  13  |  7   |  9   |  8   |

ナップザックの最大容量が10のとき, 
荷物1,3,4を入れると容量が8で価値26となります. 
総当たりで探索すると$2^N$回の探索が必要になります. (NP困難)

### QUBO式

$i$番目の荷物をナップザックに入れる場合は, $x_i=1$, 

入れない場合は, $x_i=0$とします. 

$i$番目の荷物の容量を$w_i$, 荷物の価値を$c_i$とします. 

ナップザックに入れる荷物の価値を最大化したいので, QUBOコスト関数は, 
$$
H_A = -\sum^N_{i=1} c_i x_i
$$
です. 

QUBO式にマイナスがついているのは, QUBO関数は最小に向かうために, 関数を最大化したいときにはマイナスを付けます. 

制約式は, ナップザックに入れる荷物が総体積以下になる条件
$$
\sum^N_{i=1} w_i x_i < W
$$
ですが, QUBO式は不等式を扱えません. 

そのために補助バイナリ変数$y_n$を導入します. 
$y_n$はナップザックに入れる荷物の容量の合計が, $n$のときに$y_n=1$となる変数です. 
ナップザックの最大容量は$W$なので$n$は0から$N$までの値を取ります. 

$y_n$は荷物の容量の合計値なので, $N$個ある$y_n$はただ1つのみ1になる制約があります. 
制約式を書くと, 

$$
H_B = (1-\sum^W_{n=1} y_n)^2
$$

$y_n=1$になるときに, ナップザックの荷物の容量の合計$ \sum^N_{i=1} w_i x_i = n$になるような条件式を追加します. 
$$
H_C = (\sum^W_{n=1}n y_n -  \sum^N_{i=1} w_i x_i)^2
$$

QUBO式を合計すると
$$
H = H_A + H_B +H_C
$$
$$
=-\sum^N_{i=1} c_i x_i + (1-\sum^W_{n=1} y_n)^2 + (\sum^W_{n=1}n y_n -  \sum^N_{i=1} w_i x_i)^2
$$

バイナリ変数$y_n$の節約

$y_n$はナップザックの総容量を$N$個の変数で表現したもので, 1つだけ1でその他は0になるので,変数の無駄が多くなります. 

総容量を2進数でエンコードすると$N$を表現する2進数は$log_2(W-1)+1$個の変数で実現できます. 
$$
H=-\sum^N_{i=1} c_i x_i + \sum^W_{n=1} (1-y_n)^2 + (\sum^{log_2 (W-1)}_{n=0}2^n y_n -  \sum^N_{i=1} w_i x_i)^2
$$
このエンコード方法をLOGエンコードと呼びます. 
$y_n$がただ1つ1になる最小の方法はone-hotエンコーディングと呼びます. 

## 問題設定

ナップザックの最大容量$W=12$のとき, 以下の$N=5$個の荷物を考えます. 

| 荷物 |  1   |  2   |  3   |  4   |  5   |
| ---- | ---- | ---- | ---- | ---- | ---- |
| 重さ |  3   |  4   |  6   |  1   |  5   |
| 価値 |  6   |  7   |  8   |  1   |  4   |

必要なビット変数は, $N+W=5+12=17$

In [15]:
import numpy as np

#今回は4量子ビットのみ
N=5
W=12

M=N+W
#q = Array.create('q', shape=(M), vartype='BINARY')
y = LogEncInteger("y", (0, W-1))

#重さ
w=np.array([3, 4, 6, 1, 5])

#価値
c=np.array([6 ,7, 8, 1, 4])

#式A
HA=0
for i in range(N):
    HA += -c[i]*q[i]

#式Bはlogエンコードの場合は不要
    
#式C
HC=0
for i in range(N):
    HC += w[i]*q[i]

HC=(W-HC-y)**2


#式をつなげる
H = HA + HC

Sample = {'y[0]': 0, 'q[0]': 1, 'y[1]': 0, 'q[1]': 1, 'y[2]': 0, 'q[2]': 1, 'q[3]': 0, 'q[4]': 0, 'y[3]': 0}
Cost = -20.0
